# Super Learner

## Preliminary experiments

In [1]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge, RidgeCV, Lasso, LassoCV, ElasticNetCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split, RandomizedSearchCV

from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt
import seaborn as sb
from numpy.linalg import inv

In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import pandas as pd
import numpy as np
import torch
from pathlib import Path
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models
import random

In [3]:
from pmlb import fetch_data, regression_dataset_names

## Conditionally interpretable super learner

In [84]:
def random_assignments(train_X, K=6):
    data = {'index': range(len(train_X)), 'group':  np.random.choice(K, len(train_X)) }
    df = pd.DataFrame(data)
    return df


alphas=[1e-4, 1e-3, 1e-2, 1e-1, 1, 2, 4, 8, 16, 32, 64, 132]
class BaseModel:
    def __init__(self, model_type):
        self.model_type = model_type
        self.model = self.create_model()
        if model_type not in range(1,7):
            print("model_type should be in the interval [1, 6]")

    def create_model(self):
        method_name = 'model_' + str(self.model_type)
        method = getattr(self, method_name, lambda: "nothing")
        return method()

    def model_1(self):
        return RidgeCV(cv=5, alphas=alphas)

    def model_2(self):
        return ElasticNetCV(cv=5, random_state=0, l1_ratio=0.5)

    def model_3(self):
        return ElasticNetCV(cv=5, random_state=0, l1_ratio=1)

    def model_4(self):
        return DecisionTreeRegressor(max_depth=4, max_features=0.9)

    def model_5(self):
        return DecisionTreeRegressor(max_depth=5, max_features=0.9)

    def model_6(self):
        return DecisionTreeRegressor(max_depth=6, max_features=0.9)


In [85]:
def create_base_model(train_X, train_y, m_type):
    N = train_X.shape[0]
    n = int(2.5*N/np.log(N))
    ind = np.random.choice(N, n, replace=False)
    X = train_X[ind]
    y = train_y[ind]
    base_model = BaseModel(m_type)
    base_model.model.fit(X, y)
    return base_model

In [6]:
def fit_initial_K_models(train_X, train_y, model_types):
    models = []
    N = train_X.shape[0]
    n = int(3*N/np.log(N))
    for k in range(len(model_types)):
        ind = np.random.choice(N, n, replace=False)
        X = train_X[ind]
        y = train_y[ind]
        if len(ind) > 10:
            base_model = BaseModel(model_types[k])
            base_model.model.fit(X, y)
            models.append(base_model)
    return models

In [48]:
alphas=[1e-4, 1e-3, 1e-2, 1e-1, 1, 2, 4, 8, 16, 32, 64, 132]

def fit_K_models(train_X, train_y, oracle, models, K, p=0.8):
    # sample to address overfitting 
    N = train_X.shape[0]
    ind = np.random.choice(N, int(p*N), replace=False)
    X = train_X[ind]
    y = train_y[ind]
    # assigning points using oracle
    # this will be modified 
    groups = assign_points(X, oracle)
                
    if len(groups.group.unique()) < K:
        groups, models = relabel_groups(groups, models)
        K = len(groups.group.unique())
        
    model_types = [m.model_type for m in models]
    models = []
    for k in range(len(model_types)):
        ind = groups[groups["group"] == k].index.values
        X_k = X[ind]
        y_k = y[ind]
        if len(ind) > 10:
            base_model = BaseModel(model_types[k])
            base_model.model.fit(X_k, y_k)
            models.append(base_model)
    return models

In [8]:
def compute_K_model_loss(train_X, train_y, models):
    L = []
    for i in range(len(models)):
        loss = (models[i].model.predict(train_X) - train_y)**2
        L.append(loss)
    L = np.array(L)
    return L

In [9]:
def compute_weights(L, K):
    JI_K = inv(np.ones((K, K)) - np.identity(K))
    W = []
    for i in range(L.shape[1]):
        w_i = np.matmul(JI_K, L[:,i])
        W.append(w_i)
    return np.array(W)

In [61]:
def create_extended_dataset(train_X, train_y, models, p=0.7):
    # sample to address overfitting
    K = len(models)
    N = train_X.shape[0]
    n = int(p*N)
    idx = np.random.choice(N, n, replace=False)
    X = train_X[idx]
    Y = train_y[idx]
    L = compute_K_model_loss(X, Y, models)
    W = compute_weights(L, K)
    X_ext = []
    y_ext = []
    w_ext = []
    for i in range(K):
        X_ext.append(X.copy())
        y_ext.append(i*np.ones(n))
        w_ext.append(W[:, i])
    X_ext = np.concatenate(X_ext, axis=0)
    y_ext = np.concatenate(y_ext, axis=0)
    w_ext = np.concatenate(w_ext, axis=0)
    return X_ext, y_ext, w_ext

## Neural Network oracle

In [11]:
def create_oracle_model(D_in, K, N):
    """ Returns an oracle model
    
    The size of the hidden layer is a function of the
    amount of training data
    """
    H = np.minimum(int(2*np.log(N)**2), 150)
    model = nn.Sequential(
        nn.Linear(D_in, H),
        nn.BatchNorm1d(H),
        nn.ReLU(),
        torch.nn.Linear(H, K))
    return model

In [12]:
def softmax_loss(beta, f_hat, y, w):
    y_hat = np.exp(beta*f_hat)
    den = (np.exp(beta*f_hat)).sum(axis=1)
    y_hat = np.array([y_hat[i]/den[i] for i in range(len(den))])
    loss = w*((y * (1- y_hat)).sum(axis=1))
    return loss.mean()

In [13]:
def bounded_loss(beta, y_hat, y , w):
    #y_hat = beta*y_hat
    y_hat = F.softmax(y_hat, dim=1)
    loss = (y*(1-y_hat)).sum(dim=1)
    return (w*loss).mean()

In [14]:
def train_model(model, train_dl, K, learning_rate = 0.01, epochs=100):
    beta = 1
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    KK = epochs//10 + 1
    model.train()
    for t in range(epochs):
        total_loss = 0
        total = 0
        for x, y, w in train_dl:
            x = x.cuda().float()
            y = y.cuda().long()
            w = w.cuda().float()
            y_onehot = torch.FloatTensor(y.shape[0], K).cuda()
            y_onehot.zero_()
            y_onehot = y_onehot.scatter_(1, y.unsqueeze(1), 1)
            y_hat = model(x)
            loss = bounded_loss(beta, y_hat, y_onehot , w)
       
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()*y.size(0)
            total += y.size(0)
        if t % KK == 0: print("epoch %d loss %.4f" % (t, total_loss/total))

In [15]:
def assign_points(X, oracle):
    x = torch.tensor(X).float()
    y_hat = oracle(x.cuda())
    _, pred = torch.max(y_hat, 1)
    data = {'index': range(len(X)), 'group': pred.cpu().numpy()  }
    return pd.DataFrame(data) 

In [49]:
def relabel_groups(groups, models):
    unique_models = groups.group.unique()
    old2new = {x:i for i,x in enumerate(unique_models)}
    model_subset = [models[i] for i in unique_models]
    groups.group = np.array([old2new[x] for x in groups.group.values])
    return groups, model_subset

In [17]:
from sklearn.metrics import r2_score

In [18]:
def compute_loss(X, y, oracle, models):
    oracle.eval()
    x = torch.tensor(X).float()
    y = torch.tensor(y).float()
    y_hat = oracle(x.cuda())
    _, ass = torch.max(y_hat, 1)
    preds = []
    ys = []
    k = 0
    #print(ass)
    for i in range(len(models)):
        xx = x[ass==i]
        yy = y[ass==i]
        if len(xx) > 0:
            k =+1
            pred = models[i].model.predict(xx.cpu().numpy())
            preds.append(pred)
            ys.append(yy.cpu().numpy())

    if k==1:
        preds, ys = preds[0], ys[0]
    else:
        preds = np.hstack(preds)
        ys = np.hstack(ys)
    r2 = r2_score(ys, preds)
    res = (ys - preds)**2
    return res.mean(), r2

In [19]:
def compute_single_loss(X, y, model):
    pred = model.model.predict(X)
    r2 = r2_score(y, pred)
    res = (y - pred)**2
    return res.mean(), r2

In [20]:
class OracleDataset(Dataset):
    def __init__(self, X, y, w):
        self.X = X
        self.y = y
        self.w = w
        
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx], self.w[idx]

In [21]:
def get_optimizer(model, lr = 0.01, wd = 0.0001):
    parameters = filter(lambda p: p.requires_grad, model.parameters())
    optim = torch.optim.Adam(parameters, lr=lr, weight_decay=wd)
    return optim

In [22]:
def baseline_models(train_X, train_y, valid_X, valid_y):
    best_model = None
    best_valid_r2 = 0
    best_model_type = 0
    for k in range(1,7):
        base_model = BaseModel(k)
        base_model.model.fit(train_X, train_y)
        valid_r2 = base_model.model.score(valid_X, valid_y)
        if valid_r2 > best_valid_r2:
            best_valid_r2 = valid_r2
            best_model_type = k
            best_model = base_model.model
    return best_valid_r2, best_model, [best_model_type]

## Learning rate finder

In [23]:
PATH = Path("/data2/yinterian/tmp/")
def save_model(m, p): torch.save(m.state_dict(), p)
    
def load_model(m, p): m.load_state_dict(torch.load(p))

def LR_range_finder(model, train_dl, K, lr_low=1e-5, lr_high=1, epochs=10):
    losses = []
    p = PATH/"mode_tmp.pth"
    save_model(model, str(p))
    iterations = epochs * len(train_dl)
    delta = (lr_high - lr_low)/iterations
    lrs = [lr_low + i*delta for i in range(iterations)]
    model.train()
    ind = 0
    for i in range(epochs):
        for x, y, w in train_dl:
            optimizer = get_optimizer(model, lr=lrs[ind])
            x = x.cuda().float()
            y = y.cuda().long()
            w = w.cuda().float()
            y_onehot = torch.FloatTensor(y.shape[0], K).cuda()
            y_onehot.zero_()
            y_onehot = y_onehot.scatter_(1, y.unsqueeze(1), 1)
            y_hat = model(x)
            loss = bounded_loss(beta, y_hat, y_onehot , w)
       
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            ind +=1
            losses.append(loss.item())
            
    load_model(model, str(p))
    return lrs, losses

In [24]:
def oracle_LR_range_finder(train_X, train_y, K=6):
    groups = random_assignments(train_X, K)
    models = fit_K_models(train_X, train_y, groups, model_types)
    K = len(models)
    print("models")
    X_ext, y_ext, w_ext = create_extended_dataset(train_X, train_y, models)
    train_ds = OracleDataset(X_ext, y_ext, w_ext)
    train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
    model = create_oracle_model(train_X.shape[1], K, N).cuda()
    lrs, losses = LR_range_finder(model, train_dl, K, lr_low=1e-5, lr_high=0.5)
    return lrs, losses

In [25]:
dataset = "1028_SWD"
X, y = fetch_data(dataset, return_X_y=True, local_cache_dir='/data2/yinterian/pmlb/')
train_X, test_X, train_y, test_y = train_test_split(X, y)
scaler = StandardScaler()
train_X = scaler.fit_transform(train_X)
test_X = scaler.transform(test_X)

In [26]:
X.shape

(1000, 10)

## Main 

In [27]:
lr_map = {"1028_SWD": 0.15, "1029_LEV" :0.15, "1030_ERA": 0.15, "1191_BNG_pbc": 0.02,
         "1193_BNG_lowbwt": 0.1, "1196_BNG_pharynx": 0.015, "1199_BNG_echoMonths": 0.3,
         "1203_BNG_pwLinear": 0.05, "1595_poker": 0.01, "1201_BNG_breastTumor": 0.05, "197_cpu_act": 0.2,
         "201_pol": 0.15, "215_2dplanes": 0.1, "218_house_8L": 0.05, "225_puma8NH": 0.15,
         "227_cpu_small":0.15, "294_satellite_image": 0.15, "344_mv": 0.1,
          "4544_GeographicalOriginalofMusic": 0.15, "503_wind": 0.1, "529_pollen": 0.1,
         "537_houses": 0.15, "562_cpu_small": 0.15, "564_fried": 0.1, "573_cpu_act": 0.15,
         "574_house_16H": 0.15, "583_fri_c1_1000_50": 0.15, "586_fri_c3_1000_25": 0.15 }

In [28]:
# difficult problems 5, 7, 22
dataset = "1201_BNG_breastTumor"
dataset = "1199_BNG_echoMonths"

state=2
X, y = fetch_data(dataset, return_X_y=True, local_cache_dir='/data2/yinterian/pmlb/')
train_X, test_X, train_y, test_y = train_test_split(X, y, random_state=state, test_size = 0.2)
valid_X, test_X, valid_y, test_y = train_test_split(test_X, test_y, random_state=state,
                                                            test_size =0.5) 
scaler = StandardScaler()
train_X = scaler.fit_transform(train_X)
valid_X = scaler.fit_transform(valid_X)
test_X = scaler.transform(test_X)
print(dataset, train_X.shape)

1199_BNG_echoMonths (13996, 9)


In [29]:
best_valid_r2, best_model, best_model_types = baseline_models(train_X, train_y,
                                                              valid_X, valid_y)
best_test_r2 = best_model.score(test_X, test_y)

In [30]:
best_valid_r2, best_model, best_model_types, best_test_r2

(0.4601600617324513,
 DecisionTreeRegressor(criterion='mse', max_depth=6, max_features=None,
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best'),
 [6],
 0.4460224317848724)

In [86]:
def main_loop(state, selected_datasets):
    for dataset in selected_datasets:
        learning_rate = lr_map.get(dataset, 0.15)
        X, y = fetch_data(dataset, return_X_y=True, local_cache_dir='/data2/yinterian/pmlb/')
        train_X, test_X, train_y, test_y = train_test_split(X, y, random_state=state, test_size = 0.2)
        valid_X, test_X, valid_y, test_y = train_test_split(test_X, test_y, random_state=state, test_size =0.5)
        scaler = StandardScaler()
        train_X = scaler.fit_transform(train_X)
        test_X = scaler.transform(test_X)
        valid_X = scaler.transform(valid_X)

        best_valid_r2, best_model, best_model_types = baseline_models(train_X, train_y, valid_X, valid_y)
        best_test_r2 = best_model.score(test_X, test_y)
        print("best valid R^2 %.3f best model type %d" % (best_valid_r2, best_model_types[0]))
        best_oracle = None
        best_models = [best_model] 

        #K = 6
        #groups = random_assignments(train_X, K)

        batch_size = 100000
        # number of iterations depends on the number of training points
        N = train_X.shape[0]
        N_iter = int(3000/np.log(N)**2)
        print("Number of training points %d, number iterations %d" % (N, N_iter))

        model_types = [x for x in range(1,7)]
        K = len(model_types)
        INIT_FLAG = True
        oracle = None
        for i in range(16):
            if i == 8: INIT_FLAG = True
            
            if not INIT_FLAG:
                models = fit_K_models(train_X, train_y, oracle, models, K, p=0.9)
                if len(models) == 1:
                    INIT_FLAG = True  
            
            if INIT_FLAG:
                model_types = [x for x in range(1,7)] + [1,3,6,6,6,6,6,6]
                models = fit_initial_K_models(train_X, train_y, model_types)
                INIT_FLAG = False
            
            K = len(models)
            print("Iteration %d K is %d" % (i+1, K))
            if K == 1:
                INIT_FLAG = True

            if not INIT_FLAG:
                X_ext, y_ext, w_ext = create_extended_dataset(train_X, train_y, models, p=0.9)
                train_ds = OracleDataset(X_ext, y_ext, w_ext)
                train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
                oracle = create_oracle_model(train_X.shape[1], K, N).cuda()
                train_model(oracle, train_dl, K, learning_rate, N_iter)
            
            
            if not INIT_FLAG:
                train_loss, train_r2 = compute_loss(train_X, train_y, oracle, models)
                valid_loss, valid_r2 = compute_loss(valid_X, valid_y, oracle, models)
                test_loss, test_r2 = compute_loss(test_X, test_y, oracle, models)


            print("train loss %.3f valid loss %.3f", train_loss, valid_loss)
            print("train R^2 %.3f valid R^2 %.3f", train_r2, valid_r2)
            if valid_r2 >= best_valid_r2:
                best_train_r2 = train_r2
                best_valid_r2 = valid_r2
                best_K = K
                best_models = models
                best_model_types = [m.model_type for m in models]
                best_test_r2 = test_r2 
        
        results = "dataset %s state %d K %d test ISL %.3f valid ISL %.3f model_types %s" % (
                dataset, state, len(best_models), best_test_r2, best_valid_r2, str(best_model_types))
        print(results)
        #f.write(results)
        #f.write('\n')
        #f.flush()

In [88]:
main_loop(2, ["225_puma8NH"])

best valid R^2 0.611 best model type 6
Number of training points 6553, number iterations 38
Iteration 1 K is 14
epoch 0 loss 1.0070
epoch 4 loss 0.7157
epoch 8 loss 0.6951
epoch 12 loss 0.6865
epoch 16 loss 0.6799
epoch 20 loss 0.6733
epoch 24 loss 0.6680
epoch 28 loss 0.6641
epoch 32 loss 0.6621
epoch 36 loss 0.6606
train loss %.3f valid loss %.3f 7.2258932426656495 8.139765589217903
train R^2 %.3f valid R^2 %.3f 0.6801285749411117 0.5430950989156202
Iteration 2 K is 4
epoch 0 loss 4.3251
epoch 4 loss 2.6868
epoch 8 loss 2.6024
epoch 12 loss 2.5548
epoch 16 loss 2.5392
epoch 20 loss 2.5200
epoch 24 loss 2.5083
epoch 28 loss 2.5018
epoch 32 loss 2.4942
epoch 36 loss 2.4897
train loss %.3f valid loss %.3f 9.992798919920448 13.139159047455314
train R^2 %.3f valid R^2 %.3f 0.6668817560375722 0.5661441312086251
Iteration 3 K is 2
epoch 0 loss 6.8983
epoch 4 loss 5.4638
epoch 8 loss 5.4637
epoch 12 loss 5.4637
epoch 16 loss 5.4637
epoch 20 loss 5.4637
epoch 24 loss 5.4637
epoch 28 loss 5.46

In [54]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        resnet = models.resnet18(pretrained=True)
        layers = list(resnet.children())[:8]
        self.top_model = nn.Sequential(*layers).cuda()
        self.bn = nn.BatchNorm1d(512)
        self.fc = nn.Linear(512, 10)
    
    def forward(self, x):
        x = F.relu(self.top_model(x))
        x = nn.AdaptiveAvgPool2d((1,1))(x)
        x = x.view(x.shape[0], -1) # flattening 
        x = self.bn1(x)
        x = self.fc1(x)
        return x

## Looking at results